In [ ]:
from heatmap_model.interaction_model import UQnet,TrajModel
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline 

import torch
from torch import nn, Tensor
import torchvision.datasets as dataset
from torch.optim.lr_scheduler import StepLR
import datetime

from scipy.special import expit

from heatmap_model.utils import *
from heatmap_model.train import *
from heatmap_model.interaction_dataset import *
from heatmap_model.losses import *
from config import *

from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('Worrying Stuff')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

### This notebook can reproduce the results about uncertainty quantification in the paper

- we choose the resolution = 0.5m in inference. Any resolution works, we can directly use the pre-trained models.
- Here we only show the MR minimization sampling strategy, which is the result on the INTERPRET leaderboard.

In [ ]:
paralist['resolution'] = 0.5
paralist['encoder_attention_size'] = 128
paralist['use_sem'] = False
paralist['mode'] = 'lanescore'
paralist['prob_mode'] = 'ce'
paralist['inference'] = True
model = UQnet(paralist, test=True, drivable=False).to(device) # set test=True here
testset = InteractionDataset(['val'], 'val', paralist, mode=paralist['mode'], filters=False) # for validation
#testset = InteractionDataset(['test'], 'test', paralist, mode=paralist['mode'], filters=False) # for testing

### For predicted best 6 positions and uncertainty quantification, run the following block
- <code>'val'</code> can be changed to <code>'val_all'</code> or <code>'test'</code> as needed.
- <code>Yp</code> is predicted results, <code>Ua</code> is aleatoruc uncertainty, <code>Um</code> is epistemic uncertainty, <code>Y</code> is the ground-truth

In [ ]:
Yp, Ua, Um, Y = inference_model(model, './interactionDE/', 'val', 7, testset, paralist, k=6, 
                                test=False, return_heatmap=False, mode='lanescore', batch=4)

### If you also want to visualize the heatmap results, run the following block (much slower)
- <code>H</code> is the heatmap, the other outputs are not important.

In [ ]:
M, T, Nm, Na, Yp, Ua, Um, H, Y = inference_model(model, './interactionDE/', 'val', 7, testset, paralist, 
                                              k=6, test=False, return_heatmap=True, mode=paralist['mode'], batch=4)

### An example of accuracy evaluation
- <code>r</code> is radius and <code>sh</code> is the best <code>sh</code>-th prediction

In [ ]:
FDE = ComputeError(Yp,Y, r=2, sh=6)